In [3]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import os
if torch.cuda.is_available(): device = 'cuda'
else: device = 'cpu'

In [4]:
# open text file and read in data as `text`
filename = 'Tuyen-Tap-To-Hoai-To-Hoai.txt'
with open(filename, 'r', encoding='utf-8') as f:
    text = f.read()
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])
# encoded = np.array([ch for ch in text])

In [5]:
encoded[:100]

array([131,  45, 116, 110,  50,  45, 116, 112, 112, 112,  93,  27, 118,
       121, 202, 200, 110,  37, 130, 112, 112,  93,  27, 118, 121, 202,
       200, 110, 195, 130, 112, 112,  93,  27, 118, 121, 202, 200, 110,
        48, 130, 112, 112,  93,  27, 118, 121, 202, 200, 110, 145, 130,
       112, 112,  93,  27, 118, 121, 202, 200, 110, 160, 130, 112, 112,
        93,  27, 118, 121, 202, 200, 110, 113, 130, 112, 112,  93,  27,
       118, 121, 202, 200, 110, 156, 130, 112, 112, 112, 112, 112, 112,
       147, 117, 110,  69, 200, 118, 154, 201, 110])

In [6]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

def save_checkpoint(file_name):
    model_dante = file_name+'.net'

    checkpoint = {'n_hidden': net.n_hidden,
                  'n_layers': net.n_layers,
                  'state_dict': net.state_dict(),
                  'tokens': net.chars}

    with open(model_dante, 'wb') as f:
        torch.save(checkpoint, f)

In [7]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [8]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    # start at random location within seg_length
    start = int(np.random.choice(range(seq_length),1))
    arr = arr[start:]
    
    # total number of batches we can make, // integer division, round down
    batch_size_total = batch_size * seq_length
    n_batches = len(arr)//batch_size_total

    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    
    # Reshape into batch_size rows, n. of first row is the batch size, the other lenght is inferred
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        # TODO: investigate
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y 
        
#when we call get batches we are going 
#to create a generator that iteratest through our array and returns x, y with yield command

In [9]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=612, n_layers=4,
                               drop_prob=0.5, lr=0.001, bidirectional=False):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        if bidirectional: self.D = 2
        else: self.D = 1
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # define encoder
        self.embedding = nn.Embedding(len(self.chars), n_hidden)
        
        ## TODO: define the LSTM
#         self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
#                             dropout=drop_prob, batch_first=True)
        self.lstm = nn.LSTM(n_hidden, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True, bidirectional=bidirectional)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(self.D*n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        
        word_embedded = self.embedding(x)        
        ## TODO: Get the outputs and the new hidden state from the lstm
#         r_output, hidden = self.lstm(x, hidden)
        r_output, hidden = self.lstm(word_embedded, hidden)
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.D*self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
       
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
       
        # TODO: investigate
        hidden = (weight.new(self.D*self.n_layers, batch_size, self.n_hidden).zero_().to(device),
              weight.new(self.D*self.n_layers, batch_size, self.n_hidden).zero_().to(device))
        
        return hidden

In [10]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
#         x = one_hot_encode(x, len(net.chars))

        inputs = torch.from_numpy(x)
        

        inputs = inputs.to(device)
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        # apply softmax to get p probabilities for the likely next character giving x
        p = F.softmax(out, dim=1).data
        p = p.cpu() # move to cpu
        
        # get top characters
        # considering the k most probable characters with topk method
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [11]:
def sample(net, size, prime='Cháu lên ba ', top_k=None):
        


    net.eval() # eval mode
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)
    net.train() # train mode
    return ''.join(chars)

In [12]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10, checkpoint='data.net'):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train() # set training mode for dropout and batchnorm
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    net.to(device)
    
    h = net.init_hidden(batch_size)
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        
        counter = 0
        step = len(data)//(batch_size*seq_length)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
#             x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            
            inputs, targets = inputs.to(device), targets.to(device)

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h]) # similar to detach

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
#             if counter % print_every == 0:
            if counter % step == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = 0
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
#                     x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y

                    inputs, targets = inputs.to(device), targets.to(device)

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                    val_losses = 0.9*val_losses +0.1*val_loss.item()
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}/{}...".format(counter,step),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(val_losses))
        print(sample(net, 1000, prime='Thế nào mà anh ấy bị lây cái máu dê của thằng địa đến tận giờ.', top_k=5))
        save_checkpoint(checkpoint)

In [13]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (embedding): Embedding(204, 512)
  (lstm): LSTM(512, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=204, bias=True)
)


In [14]:
batch_size = 128
seq_length = 200 #max length verses
n_epochs = 50 # start smaller if you are just testing initial behavior

# train the model
# train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=1, checkpoint=filename)

In [16]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open(filename+'.net', 'rb') as f:
    checkpoint = torch.load(f)
    
net = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
net.load_state_dict(checkpoint['state_dict'])
net.to(device)
net.eval() # eval mode
usr_input = input('Nhập 1 câu bất kỳ: ')
print('\n'+sample(net, 2000, top_k=4, prime=usr_input))

Nhập 1 câu bất kỳ: cháu lên ba

cháu lên ba mươi. Nhà tôi nói như tiếng gà giáo nó thì đi tìm đâu. Một bọn chim chít khóc nóng đến tận thế. Cái này ta đi tìm cả trưởng thôn, có chịu không biết như chúng mày còn nói đi với con chim đi chợ. Nhưng chúng tôi trông tin nằm đi thế này, tôi đã được nói với chị.

Chúng nó đi tìm chúng tôi thì đi chợ đã trở dậy. Một con trâu nhìn thấy một con gấu đã lại nghĩ ngợi như ngày trước còn nhớ chuyện những nơi chỉ còn có một nơi thì đã chạy về.

Mỗi lúc nhớ các cụ thì thào cho các bạn nhà thìa chiếc chiếc lá. Con người đã chạy đi chợ thì đi đâu. Trời đã lâu. Chỉ nghe một lúc thì người ta đi đến nghe tiếng chuông chuyện trong thành các người chết được. Trên thuyền thuê, chỉ được một câu nói cho chú con nó như thế. Nghe, chúng nó lại chết đi, nghĩ ngượng nghiêng, thế là cái giống thiết được nghĩ những chuyện đi, nhưng đi tìm được những điều nhiều chuyện cũng nghĩ thế mà chỉ thấy cái gì không biết trông thấy một người thày giáo chẳng cần có cái giống của 